In [1]:
using Pkg
Pkg.add("TestImages"); Pkg.add("Images");  Pkg.add("Colors")
using Knet, Random, IterTools, Base.Iterators, Images, Colors

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed VersionParsing ─ v1.2.0
 Installed Measures ─────── v0.3.1
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [442fdcdd] ↑ Measures v0.3.0 ⇒ v0.3.1
  [81def892] ↑ VersionParsing v1.1.3 ⇒ v1.2.0
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [2]:
function mask!(a,pad)
    x,y = size(a)
    for i = 1:x
        for j = 1:y
            if a[i, j] == pad
                a[i, j] = 0
            end
        end
    end
    return a
end

mask! (generic function with 1 method)

In [3]:
wdict = Knet.load("dictionary.jld2", "dict")
UNK = 1
EOS = 2
w2i(x) = get(wdict, x, UNK)

w2i (generic function with 1 method)

In [4]:
dataset = Knet.load("data2.jld2", "dataset")
summary(dataset)

"11532-element Array{Tuple{Array{Float32,3},Array{Int32,1}},1}"

In [5]:
y_truth = ones(Int32,length(dataset))
true_labels = ones(Int32,5952)
y_truth = y_truth[5953:end].+1
append!(true_labels, y_truth)
y_truth = copy(true_labels)
println(summary(y_truth))

rng = Random.seed!(0)
y_truth = shuffle(rng, y_truth)
rng = Random.seed!(0)
dataset = shuffle(rng, dataset)
println(summary(dataset))

9764-element Array{Int32,1}
9763-element Array{Tuple{Array{Float32,3},Array{Int32,1}},1}


In [6]:
VOCAB_SIZE = length(wdict)
BATCHSIZE = 40

40

In [7]:
mutable struct Conv 
    w 
    b 
    f_activation
    p_drop
    f_pool
end
(c::Conv)(x) = c.f_activation.(c.f_pool(conv4(c.w, dropout(x,c.p_drop)) .+ c.b))


Conv(w1::Int,w2::Int,cx::Int,cy::Int;f=relu, pdrop=0, f_pool=pool) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop, f_pool)

Conv

In [9]:
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [10]:
struct Projection; w; b; f; p; end
(d::Projection)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Projection(i::Int,o::Int,f=relu;pdrop=0) = Projection(param(o,i), param0(o), f, pdrop)

Projection

In [11]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]

In [12]:
mutable struct frame_head
    conv1
    conv2
    conv3
    conv4
    fc
    output
end

function frame_head(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims)
    conv1 = Conv(w1, w1, 2, c1)
    conv2 = Conv(w2, w2, c1, c2)
    conv3 = Conv(w3, w3, c2, c3)
    conv4 = Conv(w4, w4, c3, c4; f_pool = identity)
    fc = Dense(35840, hidden)
    output = Projection(hidden, outdims)
    frame_head(conv1, conv2, conv3, conv4, fc, output)
end
    

function (f::frame_head)(x)
    y = f.conv4(f.conv3(f.conv2(f.conv1(x))))
    println("Frame Conv: ",sum(y))
    z =f.fc(y)
    println("FullyConnect: ",sum(z))
    p = f.output(z)
    for i = 1:40
        
        println(i,"Sum: "sum(p[:,i]))
    end
end

In [13]:
mutable struct sentence_head
    embed
    encoder
end


function sentence_head(vocabsize::Int, embeddingsize::Int, hiddensize::Int)
    embed = Embed(vocabsize, embeddingsize)
    encoder = RNN(embeddingsize, hiddensize, rnnType = :lstm, h = 0)
    sentence_head(embed, encoder)
end

sentence_head

In [14]:
function (s::sentence_head)(x)
    src_embed_tensor = s.embed(x)
    s.encoder.h = 0
    s.encoder.c = 0
    y_enc = s.encoder(src_embed_tensor)
end

In [15]:
mutable struct bimodalEncoder
    fh
    sh
end


function bimodalEncoder(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims, vocabsize, embeddingsize, hiddensize)
    fh = frame_head(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims)
    sh = sentence_head(vocabsize, embeddingsize, hiddensize)
    bimodalEncoder(fh, sh)
end

function (b::bimodalEncoder)(image, label, y_truth)
    trans_nll(cosine_similarity(b, image_batch, seqbatch(labels)), y_truth)
end


function (b::bimodalEncoder)(x)

    images = []
    labels = []
   
    for i in 1:length(x)
        x1,x2 = x[i]
        
        push!(labels, x2)
        push!(images, x1)
    end
    
    image_batch = reshape(cat1d(images...), (210,160,2,length(x)))
    label_batch = permutedims(reshape(cat1d(labels...), (8, length(x))), (2,1))
    
    if (gpu()>=0)
        image_batch = KnetArray(image_batch)
    end
    
    cosine_similarity(b, image_batch, label_batch)
end



function(b::bimodalEncoder)(x,y)    
    trans_nll(b(x), y)
end

function (b::bimodalEncoder)(d::Knet.Data) 
    Knet.mean(b(x,y) for (x,y) in d)
end

In [16]:
function findmaxlength(sentences)
    maxsize = 0
    count = 0
    for sent in sentences
        count = count + 1
        if(length(sent) > maxsize)
            
            maxsize = length(sent)
        end
    end
    return maxsize
end

findmaxlength (generic function with 1 method)

In [17]:
function cosine_similarity(b::bimodalEncoder, frame_pairs, sentences)
    sum( b.sh(sentences)[:,:,end] .*  b.fh(frame_pairs), dims = 1) ./ (sqrt.(sum( b.sh(sentences)[:,:,end].^2, dims = 1)) .* sqrt.(sum( b.fh(frame_pairs).^2, dims = 1)))
end

cosine_similarity (generic function with 1 method)

In [18]:
function convertsimilaritytoprobdist(cos_similarity)
    vcat(0.5.+((cos_similarity .* sqrt.(1 .- cos_similarity.^2))./π) + (asin.(cos_similarity)./π), 1 .- (0.5.+((cos_similarity .* sqrt.(1 .- cos_similarity.^2))./π) + (asin.(cos_similarity)./π)))
end

convertsimilaritytoprobdist (generic function with 1 method)

In [19]:
function trans_nll(cos_similarity, y_truth)
    nll(convertsimilaritytoprobdist(cos_similarity),y_truth)
end

trans_nll (generic function with 1 method)

In [29]:
d = dataset[1:9000]
y = y_truth[1:9000]
dtrn = minibatch(d, y, BATCHSIZE)
dtst = minibatch(dataset[9001:end], y_truth[9001:end], BATCHSIZE)
# Knet.save("2dtrn.jld2","minibatch",dtrn)
# Knet.save("2dtst.jld2","minibatch",dtst)
summary(first(dtrn))

"Tuple{Array{Tuple{Array{Float32,3},Array{Int64,1}},1},Array{Int32,1}}"

In [21]:
# b = bimodalEncoder(5,32,5,32,4,64,3,64, 10, 10, length(wdict), 12, 10)
b = bimodalEncoder(8,64,5,64,4,128,3, 128, 100, 10, length(wdict), 12, 10)
Knet.save("bimodal2.jld2", "model", b)

bimodalEncoder(frame_head(Conv(P(Array{Float32,4}(5,5,2,32)), P(Array{Float32,4}(1,1,32,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(5,5,32,32)), P(Array{Float32,4}(1,1,32,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(4,4,32,64)), P(Array{Float32,4}(1,1,64,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(3,3,64,64)), P(Array{Float32,4}(1,1,64,1)), NNlib.relu, 0, identity), Dense(P(Array{Float32,2}(10,20160)), P(Array{Float32,1}(10)), NNlib.relu, 0), Dense(P(Array{Float32,2}(10,10)), P(Array{Float32,1}(10)), NNlib.relu, 0)), sentence_head(Embed(P(Array{Float32,2}(12,26))), LSTM(input=12,hidden=10)))

In [6]:
function accuracy(dataset, model)
    
    total_count = 0
    total_correct = 0
    for batch in dataset
        probdist = convertsimilaritytoprobdist(model(batch[1]))
        preds = vec(map(x->x[1], argmax(probdist, dims=1)))
        ground_truth = batch[2]
        total_count += length(ground_truth)
        total_correct += sum(ground_truth .== preds) 
    end
    return total_correct/total_count
end

accuracy (generic function with 1 method)

In [18]:
Random.seed!(rand(1:100000))
rand()

0.25248592638506406

In [ ]:
for i in 1:50
    global y,d
    rng = Random.seed!(rand(1:100000))
    y = shuffle(rng, y)
    d = shuffle(rng, d)
    dtrn = minibatch(d, y, BATCHSIZE)
    b = Knet.load("bimodal2.jld2", "model")
#     println("Train error before training is: ", b(dtrn))
    println("Test error before training is: ", b(dtst))
    a = adam(b, take(cycle(dtrn),500))
    progress!(a)
    println("train accuracy is: ", accuracy(dtrn,b))
    println("test accuracy is: ", accuracy(dtst, b))
    Knet.save("bimodal2.jld2", "model", b)
end

In [22]:
# # b = Knet.load("bimodal2.jld2", "model")
# println("Train error before training is: ", b(dtrn))
# println("Test error before training is: ", b(dtst))

Train error before training is: 0.7194562086190047
Test error before training is: 0.7324690840121669


In [3]:
# a = adam(b, take(cycle(dtrn),500); gclip=0.5, lr=0.0002)
# progress!(a)
@doc adam

```
minimize(func, data, optimizer=Adam(); params)
sgd     (func, data; lr=0.1,  gclip, params)
momentum(func, data; lr=0.05, gamma=0.95, gclip, params)
nesterov(func, data; lr=0.05, gamma=0.95, gclip, params)
adagrad (func, data; lr=0.05, eps=1e-6, gclip, params)
rmsprop (func, data; lr=0.01, rho=0.9, eps=1e-6, gclip, params)
adadelta(func, data; lr=1.0,  rho=0.9, eps=1e-6, gclip, params)
adam    (func, data; lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8, gclip, params)
```

Return an iterator which applies `func` to arguments in `data`, i.e.  `(func(args...) for args in data)`, and updates the parameters every iteration to minimize `func`.  `func` should return a scalar value.

The common keyword argument `params` can be used to list the `Param`s to be optimized.  If not specified, any `Param` that takes part in the computation of `func(args...)` will be updated.

The common keyword argument `gclip` can be used to implement per-parameter gradient clipping. For a parameter gradient `g`, if `norm(g) > gclip > 0`, `g` is scaled so that its norm is equal to `gclip`. If not specified no gradient clipping is performed.

These functions do not perform optimization, but return an iterator that can. Any function that produces values from an iterator can be used with such an object, e.g. `progress!(sgd(f,d))` iterates the sgd optimizer and displays a progress bar. For convenience, appending `!` to the name of the function iterates and returns `nothing`, i.e. `sgd!(...)` is equivalent to `(for x in sgd(...) end)`.

We define optimizers as lazy iterators to have explicit control over them:

  * To report progress use `progress(sgd(f,d))`.
  * To run until convergence use `converge(sgd(f,cycle(d)))`.
  * To run multiple epochs use `sgd(f,repeat(d,n))`.
  * To run a given number of iterations use `sgd(f,take(cycle(d),n))`.
  * To do a task every n iterations use `(task() for (i,j) in enumerate(sgd(f,d)) if i%n == 1)`.

These functions apply the same algorithm with the same configuration to every parameter by default. `minimize` takes an explicit optimizer argument, all others call `minimize` with an appropriate optimizer argument (see `@doc update!` for a list of possible optimizers). Before calling [`update!`](@ref) on a `Param`, `minimize` sets its `opt` field to a copy of this default optimizer if it is not already set. The `opt` field is used by the `update!` function to determine the type of update performed on that parameter.  If you need finer grained control, you can set the optimizer of an individual `Param` by setting its `opt` field before calling one of these functions. They will not override the `opt` field if it is already set, e.g. `sgd(model,data)` will perform an `Adam` update for a parameter whose `opt` field is an `Adam` object. This also means you can stop and start the training without losing optimization state, the first call will set the `opt` fields and the subsequent calls will not override them.

Given a parameter `w` and its gradient `g` here are the updates applied by each optimizer:

```
# sgd (http://en.wikipedia.org/wiki/Stochastic_gradient_descent)
w .= w - lr * g

# momentum (http://jlmelville.github.io/mize/nesterov.html)
v .= gamma * v - lr * g
w .= w + v

# nesterov (http://jlmelville.github.io/mize/nesterov.html)
w .= w - gamma * v
v .= gamma * v - lr * g
w .= w + (1 + gamma) * v

# adagrad (http://www.jmlr.org/papers/v12/duchi11a.html)
G .= G + g .^ 2
w .= w - lr * g ./ sqrt(G + eps)

# rmsprop (http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)
G .= rho * G + (1-rho) * g .^ 2 
w .= w - lr * g ./ sqrt(G + eps)

# adadelta (http://arxiv.org/abs/1212.5701)
G .= rho * G + (1-rho) * g .^ 2
update = sqrt(delta + eps) .* g ./ sqrt(G + eps)
w = w - lr * update
delta = rho * delta + (1-rho) * update .^ 2

# adam (http://arxiv.org/abs/1412.6980)
v = beta1 * v + (1 - beta1) * g
G = beta2 * G + (1 - beta2) * g .^ 2
vhat = v ./ (1 - beta1 ^ t)
Ghat = G ./ (1 - beta2 ^ t)
w = w - (lr / (sqrt(Ghat) + eps)) * vhat
```


In [ ]:
# println("Train error after training is: ", b(dtrn))
# println("Test error after training is: ", b(dtst))

In [ ]:
# function accuracy(dataset, model)
#     total_count = 0
#     total_correct = 0
#     for batch in dataset
#         probdist = convertsimilaritytoprobdist(model(batch[1]))
#         preds = vec(map(x->x[1], argmax(probdist, dims=1)))
#         ground_truth = batch[2]
#         total_count += length(ground_truth)
#         total_correct += sum(ground_truth .== preds) 
#     end
#     return total_correct/total_count
# end

In [ ]:
# Knet.save("bimodal2.jld2", "model", b)

In [ ]:
#b = Knet.load("bimodalv6.jld2", "model")

In [ ]:
#println(firstbatch[2])

In [ ]:
#Knet.save("dtrnbackup.jld2", "model", dtrn)
#Knet.save("dtstbackup.jld2", "model", dtst)

In [ ]:
#dtrn = Knet.load("dtrnbackup.jld2", "model")
#dtst = Knet.load("dtstbackup.jld2", "model")
#summary(dtst)

In [ ]:
# println("train accuracy is: ", accuracy(dtrn,b))

In [ ]:
# println("test accuracy is: ", accuracy(dtst, b))

In [ ]:
#=
sample = rand(1:1:40)
f1,f2 = first(dtst)[1][sample][1]
both_frames = hcat(f1,f2)
img = permutedims(Float32.(both_frames./255), (3,1,2))

colorview(RGB, img)

=#

In [ ]:
#first(dtst)[1][sample][2]

In [ ]:
#first(dtst)[2][sample] 
#1 true
#2 false

In [ ]:
#argmax(convertsimilaritytoprobdist(b(first(dtst)[1]))[:,sample])

In [ ]:
# Knet.save("bimodal2.jld2", "model", b)

In [19]:
a = collect(1:24)
a = reshape(a, (2,3,4))

2×3×4 Array{Int64,3}:
[:, :, 1] =
 1  3  5
 2  4  6

[:, :, 2] =
 7   9  11
 8  10  12

[:, :, 3] =
 13  15  17
 14  16  18

[:, :, 4] =
 19  21  23
 20  22  24

8×3 Array{Int64,2}:
  1   3   5
  2   4   6
  7   9  11
  8  10  12
 13  15  17
 14  16  18
 19  21  23
 20  22  24